## CIA FACTBOOK DATA ANALYSIS 

### Introduction
In this project, I will be analyzing data from the CIA World Factbook, which contains various statistics about the World and countries in it. 

The analysis will cover parameters and demographic information such as the population, population growth and surface area (land and water).

First of, allow me connect to the database and thereafter, dive in. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT 
    *
 FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


The SQLite internal table sqlite_sequence is used to store information about SQLite: autoincrement columns. This means we only have one data table, called "facts". Let's query the first five rows of the facts table.



In [3]:
%%sql
SELECT 
    * 
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [4]:
%%sql
SELECT 
    COUNT(*) AS no_of_records
 FROM facts

Done.


no_of_records
261


The table consists of 11 columns and 261 records. Of the columns, I am interested in the following:

- name - the country
- area - the country's total area (both land and water).
- area_land - the country's land area in square kilometers.
- area_water - the country's water area in square kilometers.
- population - the country's population.
- population_growth - the country's population growth as a percentage.
- birth_rate - the country's birth rate, or the number of births per year per 1,000 people.
- death_rate - the country's death rate, or the number of death per year per 1,000 people.
- migration_rate - the country's migration rate or the number of the people exiting the country per year.

### Summary Statistics

In the following analysis of the data, a couple of questions will answered to get a deeper understanding of the countries of the world. Some of those questions amongst others are:

- What are the minimum and maximum populations and minimum and maximum population growth rates?
- What are the average population and average area of all the countries?
- What countries have above average populations and which have below average areas?
- Which countries have the highest population density?
- Which countries will add the most people to their populations next year?
- Which countries have a higher death rate than birth rate?

### Minimum and maximum population and population growth

In [5]:
%%sql
SELECT 
    MIN(population) as min_pop, 
    MAX(population) as max_pop, 
    MIN(population_growth) as min_pop_grwth, 
    MAX(population_growth) as max_pop_grwth
 FROM facts

Done.


min_pop,max_pop,min_pop_grwth,max_pop_grwth
0,7256490011,0.0,4.02


In [6]:
%%sql
SELECT 
    name 
 FROM facts
 GROUP BY name
 HAVING population = (SELECT MIN(population) FROM facts)


Done.


name
Antarctica


In [7]:
%%sql
SELECT 
    name
 FROM facts
 GROUP BY name
 HAVING population = (SELECT MAX(population) FROM facts)

Done.


name
World


In [8]:
%%sql
SELECT 
    name, 
    population 
 FROM facts
 WHERE name <> 'World'
 ORDER BY population DESC
 LIMIT 2


Done.


name,population
China,1367485388
India,1251695584


In the last couple of queries, I was able to deduce the minimum and maximum population and their locations which turned out to be Antartica and World respectively. 

In an attempt not to skew our analysis, I chose to disregard the World population. Therefore, the country with the highest population is China closely followed by India.

Going forward, I will not have the World statistics in my analysis in order to get the best results.

### Average population and area

In [9]:
%%sql
SELECT ROUND(AVG(population),2) as avg_pop, 
        ROUND(AVG(area),2) as avg_area
 FROM facts
 WHERE name <> 'World'

Done.


avg_pop,avg_area
32242666.57,555093.55


### Densely populated countries (per sq km)

In [10]:
%%sql
SELECT 
    name, 
    population, 
    area
 FROM facts
 WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World')
    AND area < (SELECT AVG(area) FROM facts WHERE name <> 'World')

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


In [11]:
%%sql
SELECT 
    name, 
    population, 
    area, 
    ROUND(CAST(population as FLOAT)/area, 2) as pop_density
 FROM facts
 ORDER BY pop_density DESC
 LIMIT 15

Done.


name,population,area,pop_density
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


I have used two methods(average area vs population/area) to calculate the population density and have come to the conclusion that second one is the best way to go about it. This is so because my analysis findings from the latter method matches up finely with some external online research.  

The downside of solving population density woth the average population is that, it might exclude countries where the population is lower than average but has an extremely small land area or the land area is above average and the population is still very high. 


### Other Analyses

### Country with the highest growth rate

In [12]:
%%sql
SELECT 
    name, 
    MAX(population_growth) AS highest_growth_rt
 FROM facts

Done.


name,highest_growth_rt
South Sudan,4.02


### Top 5 countries having the highest land ratio

In [13]:
%%sql
SELECT name, 
    ROUND(CAST(area_water AS FLOAT) /area_land*100,2) as water_to_land_ratio
 FROM facts 
 ORDER BY 2 DESC
 LIMIT 5

Done.


name,water_to_land_ratio
British Indian Ocean Territory,90566.67
Virgin Islands,452.02
Puerto Rico,55.48
"Bahamas, The",38.66
Guinea-Bissau,28.47


### Countries with more water area than land area

In [14]:
%%sql
SELECT name
 FROM facts
 WHERE area_water > area_land

Done.


name
British Indian Ocean Territory
Virgin Islands


### Top 5 countries with the highest migration rate (people exiting)

In [15]:
%%sql
SELECT 
    name, 
    migration_rate
 FROM facts
 ORDER BY 2 DESC
 LIMIT 5

Done.


name,migration_rate
Qatar,22.39
American Samoa,21.13
"Micronesia, Federated States of",20.93
Syria,19.79
Tonga,17.84


### Top 5 countries with a high death to birth ratio

In [16]:
%%sql
SELECT name, ROUND(CAST(death_rate AS FLOAT) - birth_rate,2) AS death_to_birth_ratio
    FROM facts
 WHERE death_rate > birth_rate
 ORDER BY death_rate DESC
 LIMIT 5

Done.


name,death_to_birth_ratio
Ukraine,3.74
Bulgaria,5.52
Latvia,4.31
Lithuania,4.17
Russia,2.09


The above ratio tells me that of the countries that have a dwindling population due to more deaths as compared to births in a year, countries in Europe rank the highest.

### CONCLUSION

In conclusion, through the queries and questions answered, I have been able to delve into some of the World's demographic information in the CIA World Factbook. Statistics such as averges, counts and ratios were calculated on the population, area, population growth etc which allowed us arrive at certain findings. 

I was able to find out the most populated country to be China, the least populated to be Antartica, the most densely populated to be Macau and countries with more water area than land being British Indian Ocean Territory and the Virgin Island, amongst other answers arrived at.